In [74]:
#Needed to go through json
import requests_cache, requests
import pandas as pd
import urlparse
import numpy as np
import matplotlib.pyplot as plt
import requests
import requests_ftp
import requests_cache
import urllib2
from bs4 import BeautifulSoup

requests_cache.install_cache('Final_Project')

In [75]:
#Needed to scrape ONI index
import re
import itertools

In [140]:
key = "oKEFxtWBGJZobSCrwRmtcWzyXHRSJgNR"
base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"

def grab_data(station, Param, year):
    """
    Input: Station ID (str), Param (str), and year (int)
    Output: List of json objects for the given param in a given year
    """
    #Initialize the json object
    json_int = []
    
    #Send the request with the appropriate parementers
    response = requests.get(base_url, params = {
            "datasetid" : "GSOM", #Global Summary of the month
            "stationid" : station, #Station of interest
            "datatypeid" : Param, #Parameter of interest
            "units" : "standard", #Get readable units
            "startdate": str(year)+ "-01-01", #Start year in Jan
            "enddate" : str(year)+ "-12-31", #End year in Dec
            "limit" : 1000 #Get max amount of tokens
        }, headers = {"token":key});
    response.raise_for_status() # check for errors
    
    #Try to turn the above into a json. If there is no json object return void meaning to void the list later on.
    try:
        end = response.json()['results']
    except:
        end = "void"
    return end

In [141]:
#List of values of interest (temperature min, temperature max, temperature avg, and precipation)
Param =["TMIN", "TMAX", "TAVG", "PRCP"]

def cols_of_int(station, Param, Year_start, Year_end):
    """
    Input: Station of interest (str), parameter of interest(str), year start(int), and year end(int).
    Output: List of dictionaries with the date and the value of interest.
    """
    data = [] #intialize the dataframe
    
    #Loop over years to grab all years of interest in the relevant range
    for year in range(Year_start,Year_end+1):
        json_int = grab_data(station, Param, year) #Run the function listed above to grab the json object

        #If there is information run the following code:
        if json_int != "void":
            
            #Create a dictionary containing date and parameter value.
            for day in json_int:
                data_int = {"Date":"", Param:""}
                data_int[Param] = str(day["value"])
                data_int["Date"] = str(day["date"]).split("T")[0]            
                data.append(data_int)
    return data

In [142]:
def data_frame_creation(station,location,year_start,year_end):
    """
    Input: station (str), location (str), start year to collect data (int), and end year to collect data (int).
    Output: Dataframe containing all of the parameters of interest for a station.
    """
    T_min, T_max, T_avg, Precip = [cols_of_int(station,measure,year_start,year_end) for measure in Param] #Runs the previous function for all of the diferent parameters.
    Values = [T_min, T_max, T_avg, Precip] #Creates a list of all of the information
    
    Int_df = pd.DataFrame(T_min) #Sets up the dataframe using the first value
    
    #Loop going through the remaining parameters and adding the columns to the dataframe joined based on date. If no date supposed to return NA.
    for measures in Values[1:]:
        df = pd.DataFrame(measures)
        Int_df = pd.merge(Int_df, df, how='outer', on='Date')
        Int_df["station"] = station #Add the station information
        Int_df["location"] = location #Add the station information
    return Int_df

In [11]:
#Location of all the domestic station id's and locations of interest
domestic_station_ids = [['GHCND:USW00023271','Sacramento, CA'],['GHCND:USW00022521','Honolulu, HI'],['GHCND:USW00024233','Seattle, WA'],['GHCND:USW00012839','Miami, FL'],['GHCND:USW00023174','Los Angeles, CA'],['GHCND:USW00014733','Buffalo, NY'],['GHCND:USW00093037','Colorado Springs, CO'],['GHCND:USW00013988','Kansas City, MO'],['GHCND:USW00024229','Portland, OR'],['GHCND:USC00111577','Chicago, IL'],['GHCND:USW00012930','New Orleans, LA'],['GHCND:USW00014922','Minneapolis, MN'],['GHCND:USW00025309','Juneau, AK'],['GHCND:USW00013897','Nashville, TN'],['GHCND:USC00186350','Washington D.C.'],['GHCND:USW00014606','Bangor, ME']]

#If we are interested in international stations this is where they are located
international_station_ids = [['GHCND:JA000047891','Takamatsu, JPN'],['GHCND:RP000098430','Manila, PHI'],['GHCND:ASN00066062','Sydney, AUS'],['GHCND:NZ000933090','New Plymouth, NZ'],['GHCND:AR000087217','La Rioja, ARG'],['GHCND:CO000080342','Pasto, COL'],['GHCND:CSM00078762','San Jose, COS'],['GHCND:MXM00076903','Tapachula, MX'],['GHCND:AYM00089055','Base Marambio, AY'],['GHCND:CA002401200','Eureka Station, NW CAN']]

In [41]:
#This is the code that takes in all the stations of interest and goes throught he 3 functions above.
#THIS CODE TAKES ABOUT 30 min to run!!!!
list_data = []
for x in domestic_station_ids:
    list_data.append(data_frame_creation(x[0],x[1],1950,2016))

In [98]:
#Combine the seperate dataframes in the list into a signle dataframe. Ignored index for easier use later on.
final_df = pd.concat(list_data, ignore_index=True)

#This code adds the month and year on to the data frame
final_df['Month'] = [month.split("-")[1] for month in final_df['Date']]
final_df['Year'] = [year.split("-")[0] for year in final_df['Date']]

In [99]:
#This is the df before the ONI index
final_df

,Date,TMIN,TMAX,station,location,TAVG,PRCP,Month,Year
0,1950-01-01,36.6,50.4,GHCND:USW00023271,"Sacramento, CA",43.5,4.41,01,1950
1,1950-02-01,41.8,61.3,GHCND:USW00023271,"Sacramento, CA",51.6,3.27,02,1950
2,1950-03-01,43.5,63.6,GHCND:USW00023271,"Sacramento, CA",53.5,2.0,03,1950
3,1950-04-01,49.3,74.6,GHCND:USW00023271,"Sacramento, CA",62.0,1.04,04,1950
4,1950-05-01,51.1,80.5,GHCND:USW00023271,"Sacramento, CA",65.8,0.37,05,1950
5,1950-06-01,54.5,84.8,GHCND:USW00023271,"Sacramento, CA",69.7,0.05,06,1950
6,1950-07-01,57.7,93.4,GHCND:USW00023271,"Sacramento, CA",75.6,0.01,07,1950
7,1950-08-01,59.7,93.4,GHCND:USW00023271,"Sacramento, CA",76.5,0.0,08,1950
8,1950-09-01,58.3,85.0,GHCND:USW00023271,"Sacramento, CA",71.6,0.62,09,1950
9,1950-10-01,53.1,75.7,GHCND:USW00023271,"Sacramento, CA",64.4,2.35,10,1950


Extract data for the ONI index

In [139]:
#This section scrapes the ONI index page
url = "http://www.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/ensoyears.shtml"
ONI_req = requests.get(url) #Ping the server
ONI_html = ONI_req.text #Obtain the text file of the webstie
ONI = BeautifulSoup(ONI_html, 'lxml') #Change to html code
ONI_data = ONI.findAll('table',{'border':"1"})[0]

In [144]:
#This section creates the ONI matrix.
test = ONI_data.findAll('td', {'width':'7%'})
data = []

#Eliminate the unneeded \n's and the rows containing the months found every so often in the data 
for i in test:
    value = str(i.text).strip("\n")
    if re.search("[A-z]", value, re.IGNORECASE) == None:
        data.append(value)

#Put the data into a dataframe        
subList = [data[n:n+13] for n in range(0, len(data), 13)]
col = ["Year", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
data_complete = pd.DataFrame(subList, columns = col)
data_complete1 = data_complete.set_index("Year")
data_complete

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1950,-1.4,-1.2,-1.1,-1.2,-1.1,-0.9,-0.6,-0.6,-0.5,-0.6,-0.7,-0.8
1,1951,-0.8,-0.6,-0.2,0.2,0.2,0.4,0.5,0.7,0.8,0.9,0.7,0.6
2,1952,0.5,0.4,0.4,0.4,0.4,0.2,0,0.1,0.2,0.2,0.2,0.3
3,1953,0.5,0.6,0.7,0.7,0.7,0.7,0.7,0.7,0.8,0.8,0.8,0.7
4,1954,0.7,0.4,0,-0.4,-0.5,-0.5,-0.5,-0.7,-0.7,-0.6,-0.5,-0.5
5,1955,-0.6,-0.6,-0.7,-0.7,-0.7,-0.6,-0.6,-0.6,-1.0,-1.4,-1.6,-1.4
6,1956,-0.9,-0.6,-0.6,-0.5,-0.5,-0.4,-0.5,-0.5,-0.4,-0.4,-0.5,-0.4
7,1957,-0.3,0,0.3,0.6,0.7,0.9,1.0,1.2,1.1,1.2,1.3,1.6
8,1958,1.7,1.5,1.2,0.8,0.7,0.6,0.5,0.4,0.4,0.5,0.6,0.6
9,1959,0.6,0.5,0.4,0.2,0.1,-0.2,-0.3,-0.3,-0.1,-0.1,-0.1,-0.1


In [143]:
#This section creates the Date column of the ONI index with the proper date format.
years_with_months = []
for year in data_complete['Year']:
    for num in range(1,13):
        years_no_months = np.repeat(year,12) #Need a year start for each month.
        if num <10: #Want the 0 in front of the number if it is a single digit.
            years_with_months.append(years_no_months[num-1] + '-0' +str(num)+'-01')
        else:  #Don't want the 0 in front of the number if it is a single digit.
            years_with_months.append(years_no_months[num-1] + '-' +str(num)+'-01')

In [145]:
del data[0::13]

In [146]:
d = {'Date': years_with_months, 'ONI': data}
ONI_df = pd.DataFrame(d)
final_df_ONI = pd.merge(final_df, ONI_df[:-12], how='outer', on='Date')
final_df_ONI

,Date,TMIN,TMAX,station,location,TAVG,PRCP,Month,Year,ONI
0,1950-01-01,36.6,50.4,GHCND:USW00023271,"Sacramento, CA",43.5,4.41,01,1950,-1.4
1,1950-01-01,66.1,78.5,GHCND:USW00022521,"Honolulu, HI",72.3,13.24,01,1950,-1.4
2,1950-01-01,19.1,30.7,GHCND:USW00024233,"Seattle, WA",24.9,8.92,01,1950,-1.4
3,1950-01-01,67.3,78.9,GHCND:USW00012839,"Miami, FL",73.1,0.5,01,1950,-1.4
4,1950-01-01,40.3,56.7,GHCND:USW00023174,"Los Angeles, CA",48.5,2.54,01,1950,-1.4
5,1950-01-01,24.9,44.0,GHCND:USW00014733,"Buffalo, NY",34.4,5.02,01,1950,-1.4
6,1950-01-01,17.1,45.5,GHCND:USW00093037,"Colorado Springs, CO",31.3,0.32,01,1950,-1.4
7,1950-01-01,19.6,41.1,GHCND:USW00013988,"Kansas City, MO",30.3,0.69,01,1950,-1.4
8,1950-01-01,21.2,32.8,GHCND:USW00024229,"Portland, OR",27.0,10.11,01,1950,-1.4
9,1950-01-01,19.8,38.9,GHCND:USC00111577,"Chicago, IL",29.4,3.78,01,1950,-1.4


Stuff I didn't need for the above but that might be useful.

In [7]:
#Check if the station satisfies the necessary condition to be in my function
#Function was slow for me at the end. If we want to add more stations run them through this.
for stations in station_ids[-2:]:
    for value in Param:
        if cols_of_int(stations[0],stations[1],value,2010,2016) == []:
            print value + "," + stations[0] + "," + stations[1] + " bad"
        else:
            print value + "," + stations[0] + "," + stations[1] + " good"

TMIN,GHCND:AYM00088963,Base Esperanza, AY good
TMAX,GHCND:AYM00088963,Base Esperanza, AY good
TAVG,GHCND:AYM00088963,Base Esperanza, AY bad
PRCP,GHCND:AYM00088963,Base Esperanza, AY good
TMIN,GHCND:CA002401200,Eureka Station, NW CAN good
TMAX,GHCND:CA002401200,Eureka Station, NW CAN good
TAVG,GHCND:CA002401200,Eureka Station, NW CAN good


KeyboardInterrupt: 

In [147]:
#This might be useful if we need a loction for the station id
def station_data(station_id):
    """
    Input station ID as string (ex)"GHCND:USC00042294")
    Returns dictionary containing station location, station ID, recording start date
    """
    url_station = "https://www.ncdc.noaa.gov/cdo-web/datasets/GHCND/stations/"+station_id+"/detail"
    station_req = requests.get(url_station) #Ping the server
    station_text = station_req.text #Obtain the text file of the webstie
    station = BeautifulSoup(station_text, 'lxml') #Change to html code
    body_text = station.find_all('td',attrs={'class':'val'})
    station_info = {"Location": "" , "Station_id": ""}
    station_info["Location"] = body_text[0].text
    station_info["Station_id"] = station_id
    return station_info